# Prediction of leaderboard score (R)

In [1]:
score_data = read.csv('../input/scores.csv')
score_data[with(score_data, order(leaderboard_score)), ]

,model,leaderboard_score,accuracy,logloss,AUC,f1,mu,std
19,LogisticRegression,0.4411,0.7708,7.9152,0.554,0.2235,0.7601,0.0013
3,GradientBoostingClassifier,0.4452,0.7934,7.1356,0.6309,0.4251,0.7538,0.0047
20,LogisticRegressionCV,0.4457,0.783,7.4954,0.5794,0.2938,0.7602,0.0012
1,nolearn,0.4566,0.8056,6.7159,0.6711,0.5044,NA,NA
4,ExtraTreesClassifier,0.4729,0.776,7.7353,0.5996,0.3582,0.7526,0.0061
5,XGBClassifier,0.4851,0.7743,7.7953,0.6034,0.3689,0.7474,0.0065
13,BaggingClassifier,0.4885,0.7743,7.7952,0.5985,0.3564,0.7523,0.0051
2,ensemble of averages,0.4896,NA,NA,NA,NA,NA,NA
11,SVC,0.5336,0.7552,8.4548,0.5264,0.1455,0.742,0.0079
6,SGDClassifier,0.567,0.7274,9.4143,0.5528,0.2765,0.6339,0.069


# missing mu & std get the column means

In [2]:
# for(i in 7:8){
#   score_data[is.na(score_data[,i]), i] <- mean(score_data[,i], na.rm = TRUE)
# }
# score_data

# Model using all variables

In [6]:
lm.fit = lm(leaderboard_score ~ accuracy + logloss + AUC + f1 + mu + std, 
            data      = score_data, 
            na.action = na.omit)

slm.fit = step(lm.fit, direction = "both")
summary(slm.fit)

Start:  AIC=-33.97
leaderboard_score ~ accuracy + logloss + AUC + f1 + mu + std

           Df Sum of Sq     RSS     AIC
- f1        1   0.01384 0.23411 -35.242
- AUC       1   0.02905 0.24932 -34.487
<none>                  0.22027 -33.974
- accuracy  1   0.15164 0.37191 -29.688
- logloss   1   0.15400 0.37427 -29.612
- std       1   0.17145 0.39172 -29.065
- mu        1   0.40484 0.62511 -23.457

Step:  AIC=-35.24
leaderboard_score ~ accuracy + logloss + AUC + mu + std

           Df Sum of Sq     RSS     AIC
<none>                  0.23411 -35.242
+ f1        1   0.01384 0.22027 -33.974
- accuracy  1   0.18442 0.41853 -30.271
- logloss   1   0.18647 0.42059 -30.212
- std       1   0.19944 0.43355 -29.848
- AUC       1   0.29279 0.52690 -27.508
- mu        1   0.42450 0.65861 -24.830



Call:
lm(formula = leaderboard_score ~ accuracy + logloss + AUC + mu + 
    std, data = score_data, na.action = na.omit)

Residuals:
      Min        1Q    Median        3Q       Max 
-0.246576 -0.066364 -0.004653  0.067846  0.271408 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)  
(Intercept) -7454.1877  3419.8922  -2.180   0.0721 .
accuracy     7439.8570  3422.1310   2.174   0.0727 .
logloss       216.3608    98.9711   2.186   0.0715 .
AUC             9.1473     3.3393   2.739   0.0338 *
mu              2.9374     0.8905   3.298   0.0164 *
std           -11.4574     5.0678  -2.261   0.0645 .
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.1975 on 6 degrees of freedom
  (10 observations deleted due to missingness)
Multiple R-squared:  0.8708,	Adjusted R-squared:  0.7632 
F-statistic:  8.09 on 5 and 6 DF,  p-value: 0.01214


# Remove mu & std (b/c of missing values)

In [7]:
lm.fit = lm(leaderboard_score ~ accuracy + logloss + AUC + f1, 
            data      = score_data, 
            na.action = na.omit)

slm.fit = step(lm.fit, direction = "both")
summary(slm.fit)

Start:  AIC=-28.95
leaderboard_score ~ accuracy + logloss + AUC + f1

           Df Sum of Sq     RSS     AIC
- f1        1   0.08791 0.95445 -29.599
- AUC       1   0.09190 0.95844 -29.541
<none>                  0.86654 -28.952
- accuracy  1   0.69708 1.56362 -22.689
- logloss   1   0.69817 1.56471 -22.679

Step:  AIC=-29.6
leaderboard_score ~ accuracy + logloss + AUC

           Df Sum of Sq     RSS     AIC
- AUC       1   0.00592 0.96037 -31.513
<none>                  0.95445 -29.599
+ f1        1   0.08791 0.86654 -28.952
- accuracy  1   0.61433 1.56878 -24.643
- logloss   1   0.61516 1.56961 -24.635

Step:  AIC=-31.51
leaderboard_score ~ accuracy + logloss

           Df Sum of Sq     RSS     AIC
<none>                  0.96037 -31.513
+ AUC       1   0.00592 0.95445 -29.599
+ f1        1   0.00193 0.95844 -29.541
- accuracy  1   0.72206 1.68243 -25.663
- logloss   1   0.72259 1.68296 -25.659



Call:
lm(formula = leaderboard_score ~ accuracy + logloss, data = score_data, 
    na.action = na.omit)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.35224 -0.14958 -0.06438  0.11867  0.58624 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)  
(Intercept) -10466.7     3639.5  -2.876   0.0151 *
accuracy     10465.4     3639.1   2.876   0.0151 *
logloss        303.2      105.4   2.877   0.0151 *
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.2955 on 11 degrees of freedom
  (8 observations deleted due to missingness)
Multiple R-squared:  0.4852,	Adjusted R-squared:  0.3916 
F-statistic: 5.183 on 2 and 11 DF,  p-value: 0.02595


In [5]:
score_data = score_data[with(score_data, order(leaderboard_score)), ]

library(knitr)
foo = kable(score_data, format = "markdown", digits = 4)
foof = ''
for (i in 1:length(foo)) {
    subs = substr(foo[i],5,52)
    foof = cat(foof,cat(subs,'\n'))
}
foof

|model                      | leaderboard_score| 
|:--------------------------|-----------------:| 
|LogisticRegression         |            0.4411| 
|GradientBoostingClassifier |            0.4452| 
|LogisticRegressionCV       |            0.4457| 
|nolearn                    |            0.4566| 
|ExtraTreesClassifier       |            0.4729| 
|XGBClassifier              |            0.4851| 
|BaggingClassifier          |            0.4885| 
|ensemble of averages       |            0.4896| 
|SVC                        |            0.5336| 
|SGDClassifier              |            0.5670| 
|cosine_similarity          |            0.5732| 
|KMeans                     |            0.6289| 
|AdaBoostClassifier         |            0.6642| 
|KNeighborsClassifier       |            1.1870| 
|RandomForestClassifier     |            1.7907| 
|voting_ensemble_hard       |                NA| 
|voting_ensemble_soft       |                NA| 
|bagged_gbc                 |                NA| 


NULL